In [1]:
%pip install PuLP

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 37.2 MB/s eta 0:00:00


In [3]:
"""
Simplified Crude Trading Optimization Model (May Only) with June Premium for WTI
and Immediate Sale for WTS

Assumptions:
-----------------------------
WTI:
- Maximum daily purchase: 100,000 bbl/day over May → 3,100,000 bbl total.
- Maximum available for WTI is 80% of that: 2,480,000 bbl.
- Purchase price in Midland = $70.00/bbl.
- Pipeline (committed) cost: $0.55 + (0.2% * $70.00) ≈ $0.55 + $0.14 = $0.69/bbl.
- Storage cost (allocated) = ~$0.33/bbl.
  → Total cost per barrel = 70.00 + 0.69 + 0.33 = $71.02.
- June sale price in Houston = $71.45/bbl.
  → Net margin for WTI = 71.45 – 71.02 = $0.43/bbl.

WTS:
- Maximum available for WTS is 20% of total: 620,000 bbl.
- Purchase price in Midland = $70.00 + (-$1.00) = $69.00/bbl.
- Pipeline cost (committed) = $0.55 + (0.2% * $70.00) = $0.69/bbl.
  → Total cost per barrel = 69.00 + 0.69 = $69.69/bbl.
- Houston sale price = $70.65 + (-$0.75) = $69.90/bbl.
  → Net margin for WTS = 69.90 – 69.69 = $0.21/bbl.

There is no explicit requirement to be net flat at the end of May,
so leftover positions (if any) are not forced to be hedged with June futures.
We assume the operator will use all available storage for WTI and sell it in June,
capturing the extra margin.

Objective: Maximize total gross operating margin in May:
    0.43 * (WTI barrels) + 0.21 * (WTS barrels)

Author: (Your Name)
Date:   (Today's Date)
"""

import pulp
import matplotlib.pyplot as plt

###############################################################################
# 1) DATA & PARAMETERS
###############################################################################

days_in_may = 31

# Prices ($/bbl)
P_WTI_midland = 70.00         # Purchase price for WTI in Midland in May
P_WTI_houston_May = 70.65     # Houston sale price for WTI in May (if sold immediately)
P_WTI_houston_June = 71.45    # Houston sale price for WTI in June

P_WTS_midland = 70.00 - 1.00   # 69.00 $/bbl
P_WTS_houston = 70.65 - 0.75   # 69.90 $/bbl

# Pipeline cost ($/bbl) for both streams (committed)
pipeline_fee = 0.55
pipeline_loss = 0.002 * P_WTI_midland  # 0.002 * 70 = 0.14
Pipeline_cost = pipeline_fee + pipeline_loss   # ≈ 0.69 $/bbl

# Storage cost for WTI (allocated, assumed)
Storage_cost = 0.33           # $/bbl

# Effective cost per barrel:
Cost_WTI = P_WTI_midland + Pipeline_cost + Storage_cost   # 70 + 0.69 + 0.33 = 71.02 $/bbl
Margin_WTI = P_WTI_houston_June - Cost_WTI                 # 71.45 - 71.02 = 0.43 $/bbl

Cost_WTS = P_WTS_midland + Pipeline_cost   # 69.00 + 0.69 = 69.69 $/bbl
Margin_WTS = P_WTS_houston - Cost_WTS      # 69.90 - 69.69 = 0.21 $/bbl

# Purchase constraints:
max_daily = 100_000         # barrels per day
max_may_total = days_in_may * max_daily  # 3,100,000 barrels in May

# Maximum available for each quality:
max_WTI = 0.80 * max_may_total   # 80% for WTI = 2,480,000 barrels
max_WTS = 0.20 * max_may_total   # 20% for WTS = 620,000 barrels

###############################################################################
# 2) BUILD THE MODEL
###############################################################################
model = pulp.LpProblem("May_Optimization_with_June_Premium", pulp.LpMaximize)

###############################################################################
# 3) DECISION VARIABLES
###############################################################################
# WTI purchase in May (barrels)
X_WTI = pulp.LpVariable("X_WTI", lowBound=0, upBound=max_WTI)

# WTS purchase in May (barrels)
X_WTS = pulp.LpVariable("X_WTS", lowBound=0, upBound=max_WTS)

# Total purchase must not exceed total capacity
model += (X_WTI + X_WTS) <= max_may_total, "Total_Purchase_Cap"

# For WTI, assume all purchased is piped to Houston and stored.
# We force the pipeline volume and storage to equal the purchase.
Y_WTI_pipe = pulp.LpVariable("Y_WTI_pipe", lowBound=0)
model += (Y_WTI_pipe == X_WTI), "WTI_Pipeline_Flow"

# For WTI storage in Houston, we assume that all piped WTI is stored.
I_WTI_Hou = pulp.LpVariable("I_WTI_Hou", lowBound=0, upBound=0.80 * max_may_total)
model += (I_WTI_Hou == Y_WTI_pipe), "WTI_Storage_Flow"

# In our model, all WTI stored in Houston will be sold in June.
# We do not model the June sale explicitly, but we incorporate the extra margin in the objective.
# For WTS, assume immediate sale in May.
Y_WTS_pipe = pulp.LpVariable("Y_WTS_pipe", lowBound=0)
model += (Y_WTS_pipe == X_WTS), "WTS_Pipeline_Flow"

# Let Z_WTS_May be the barrels of WTS sold in Houston in May.
Z_WTS_May = pulp.LpVariable("Z_WTS_May", lowBound=0)
model += (Z_WTS_May == Y_WTS_pipe), "WTS_Sale_May"

###############################################################################
# 4) OBJECTIVE FUNCTION
###############################################################################
# For WTI, profit per barrel is $0.43 (if stored and sold in June)
Profit_WTI = Margin_WTI * X_WTI

# For WTS, profit per barrel is $0.21 (if sold immediately in May)
Profit_WTS = Margin_WTS * X_WTS

# Total gross operating margin (May) is the sum:
model += Profit_WTI + Profit_WTS, "Maximize_GOM"

###############################################################################
# 5) SOLVE THE MODEL
###############################################################################
solution = model.solve(pulp.PULP_CBC_CMD(msg=0))
print("==================================================")
print("SOLVER STATUS:")
print("==================================================")
print("Status:", pulp.LpStatus[model.status])
print()

print("==================================================")
print("OPTIMAL OBJECTIVE VALUE (Gross Operating Margin for May):")
print("==================================================")
print(f"${pulp.value(model.objective):,.2f}")
print()

###############################################################################
# 6) PRINT DECISION VARIABLES & IMPORTANT NUMBERS
###############################################################################
print("==================================================")
print("DECISION VARIABLES:")
print("==================================================")
print(f"WTI Purchase (May)       = {X_WTI.varValue:,.0f} barrels")
print(f"WTS Purchase (May)       = {X_WTS.varValue:,.0f} barrels")
print(f"Total Purchase           = {(X_WTI.varValue + X_WTS.varValue):,.0f} barrels")
print(f"WTI Pipeline Volume      = {Y_WTI_pipe.varValue:,.0f} barrels (should equal WTI purchase)")
print(f"WTI Houston Storage      = {I_WTI_Hou.varValue:,.0f} barrels (max 80% of total = {0.80*max_may_total:,.0f})")
print(f"WTS Pipeline Volume      = {Y_WTS_pipe.varValue:,.0f} barrels (should equal WTS purchase)")
print(f"WTS Sold in May          = {Z_WTS_May.varValue:,.0f} barrels")
print()

print("==================================================")
print("Key Calculations:")
print("==================================================")
print(f"WTI Purchase Price       = ${P_WTI_midland:.2f}/bbl")
print(f"WTI Pipeline Cost        = ${Pipeline_cost:.2f}/bbl")
print(f"WTI Storage Cost         = ${Storage_cost:.2f}/bbl")
print(f"Effective WTI Cost        = ${Cost_WTI:.2f}/bbl")
print(f"June Sale Price for WTI   = ${WTI_houston_June:.2f}/bbl")
print(f"WTI Net Margin           = ${Margin_WTI:.2f}/bbl")
print()
print(f"WTS Purchase Price       = ${P_WTS_midland:.2f}/bbl")
print(f"WTS Pipeline Cost        = ${Pipeline_cost:.2f}/bbl")
print(f"Effective WTS Cost        = ${Cost_WTS:.2f}/bbl")
print(f"Houston Sale Price for WTS= ${P_WTS_houston:.2f}/bbl")
print(f"WTS Net Margin           = ${Margin_WTS:.2f}/bbl")
print()

print("==================================================")
print("Calculated Gross Operating Margin:")
print("==================================================")
print(f"WTI GOM (May->June): ${Margin_WTI * X_WTI.varValue:,.2f}")
print(f"WTS GOM (May):      ${Margin_WTS * X_WTS.varValue:,.2f}")
print(f"Total GOM:          ${pulp.value(model.objective):,.2f}")

###############################################################################
# 7) OPTIONAL VISUALIZATIONS
###############################################################################
plt.figure()
plt.bar(["WTI Purchase", "WTS Purchase"], [X_WTI.varValue, X_WTS.varValue])
plt.title("May Purchase Volumes (Barrels)")
plt.ylabel("Barrels")
plt.show()


SOLVER STATUS:
Status: Optimal

OPTIMAL OBJECTIVE VALUE (Gross Operating Margin for May):
$1,196,600.00

DECISION VARIABLES:
WTI Purchase (May)       = 2,480,000 barrels
WTS Purchase (May)       = 620,000 barrels
Total Purchase           = 3,100,000 barrels
WTI Pipeline Volume      = 2,480,000 barrels (should equal WTI purchase)
WTI Houston Storage      = 2,480,000 barrels (max 80% of total = 2,480,000)
WTS Pipeline Volume      = 620,000 barrels (should equal WTS purchase)
WTS Sold in May          = 620,000 barrels

Key Calculations:
WTI Purchase Price       = $70.00/bbl
WTI Pipeline Cost        = $0.69/bbl
WTI Storage Cost         = $0.33/bbl
Effective WTI Cost        = $71.02/bbl


NameError: name 'WTI_houston_June' is not defined